# Mobile Robotics Project Report
### Authors: Victor David S Dubois (344303), Luca Jiménez Glur (345448), Loïc Tronchet (341181), Maria de Lluch Maqueo Colom (374820)


## Introduction
The project consists of controlling a Thymio robot allowing it to follow a path created through global mapping as well as avoiding spontaneous obstacles.  

The specific objectives and how they were achieved are described below. 

*Thymio must avoid obstacles through global navigation, that is, without relying on the sensors.*

- The A* algorithm was chosen to create the optimal path for the Thymio to reach its target because it effectively balances between computational efficiency and optimality. Unlike Dijkstra's algorithm, A* explores fewer nodes depending on the value of the function combining the motion and the heuristic cost.

*Thymio must go from an arbitrary position to a target, both of which can be moved to see how the system performs.*

- The camera gives the position and the orientation of the robot by identifying the magenta and the cyan dots on top of the Thymio robot. These positions will be sent to the A* Algorithm which can effectively create an optimal path. Two different colored dots were chosen because this way they can be easily identified due to their hue and saturation, and the orientation of the robot can be obtained. 

*Robot position must be estimated and plotted. Create a map where we can see the position of the robot compared to its global plan.*

- A window is created where the camera image can be seen. Within this picture, the planned path is illustrated, in addition to the live tracking of the robot by the camera and the kalman filter position estimation. 
  
*Thymio must be able to use local navigation avoiding obstacles placed in its path at any point in time.*

- A neural network is used to avoid the local obstacles, changing the velocity of the motors when an object is sensed by one of the seven sensors. Assigning weights allows the motor velocity to be adjusted based on the direction by which the robot is approaching the obstacle. This method was preferred over other local avoidance techniques because it is not computationally expensive and it can be tuned to define how aggressive the behavior is. 
  
*If the robot is kidnapped, that is, placed in a new starting position, the new pose should be estimated.*

- Robot kidnapping is detected through the ground sensors and the new optimal path is calculated once the robot is placed on the ground. Through the camera, the new position is obtained. 
  
*If the camera is hidden, the robot should continue its path relying solely on relative sensing.*

- The Kalman filter depends on a flag passed by the camera which indicates whether the camera measurements should be used or not. This behavior allows the filter to operate with or without the visual input, that is, only relying on the estimations calculated from the motor velocities.   


### Environment Chosen
The map was chosen to be a 9x6 grid with squares of 140 mm x 140 mm, chosen because in this manner the Thymio would fit with a reasonable margin on each side. Considering the width of the Thymio is approximately 11 cm, 140 mm would give it a margin of 15 mm on each side.

The obstacles in the map were strategically placed to allow the Thymio to have at least one path where it had to take several turns. Overcrowding with global obstacles was avoided to allow local obstacles to be added and observe the robot’s behavior. The final obstacle distribution was a result of trial and errors that produced the figure below.  

The grid representing the map was created using Python code and Chat GPT. The coordinate system was chosen to have its origin (x=0,y=0) in the center of the upper left corner square of the grid. The x axis goes along the width of the grid and increases to the right. The y axis goes along the length of the grid and increases when going down. Due to this choice, it made sense to have angles that increase when going clockwise and an orientation of 0 would correspond to pointing in the increasing x direction (that is, to the right). This description has been summarized in the figure below.

<div style="display: flex; justify-content: space-around; align-items: center;">

  <figure style="text-align: center;">
    <img src="img\grid_picture.jpg" alt="Figure 1" style="width: 80%;"/>
    <figcaption>Figure 1</figcaption>
  </figure>

</div>

In the image, the white cells represent free space while the black cells are obstacles which must be avoided. These squares can be easily detected with the camera due to the high contrast, however, the sensors do not detect them allowing a finite state machine to be implemented whose condition that switches between local and global navigation is triggered by the sensing of an obstacle. 

The target the Thymio must reach is represented by a cyan colored square to easily identify with the camera and send its coordinates to the A* algorithm. A grid was chosen as the base because in this manner a finite number of cells would be extracted. The obstacles can fit in one of the cells and be aligned with the grid. In the corners of the grid, squares are placed in order to accurately extract the lines and warp the image correctly. Local obstacles are cylindrical pieces that do not occupy the entire cell to allow the Thymio to properly avoid them without hitting them. 

## Code Implementation

In [1]:
!pip install tqdm scipy

In [2]:
#The exclamation point (!) allow to execute a terminal command in the notebook:
!python -m tdmclient list

id:         8317b8c1-1e42-4de6-ae90-56fa4505fd94
group id:   72650c50-f991-4c4a-91e2-c87be6abea9b
type:       1 (Thymio II wireless)
product id: 8 (Thymio II)
name:       Thymio 604
status:     2 (available)
cap:        3
firmware:   14



In [3]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 8317b8c1-1e42-4de6-ae90-56fa4505fd94

In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from heapq import heappush, heappop # used if you want to use heap queue for priority queue (easier to implement) but not compulsory
from scipy.signal import find_peaks
from tqdm import tqdm
import time
import sys

## 1. Vision Processing
The vision system takes the images captured by the camera and locates the robot, obstacles, goal and start position. 

To achieve this, the vision is divided into __four main tasks__: 
1. Detect the map and transform the image to a full-frame top-down view of the map.
2. Detect the obstacles
3. Detect the goal
4. Locate the robot
### Detailed explanation of each task:
1. #### Map Detection and Perspective Transformation  
To detect the map, we rely on four distinct squares, one positioned in each corner. In the HSV color space, the color of these squares is significantly different from the other colors in the image. After smoothing the image with a Gaussian filter, by calculating the difference in Hue values between each pixel and the reference color, we generate a mask. In this case, the reference Hue value is so distinct that we can depend solely on the Hue channel.  
Using this mask, contours can be easily detected and classified as squares or not based on their area and the polygon they most closely match. Once four squares are identified, the algorithm determines their centroids and edge lengths.  
To isolate the map from the rest of the frame, we perform a perspective transformation. This process uses the four computed centroids, relocating the points to align with the map's corners. The OpenCV function *getPerspectiveTransform()* is used to calculate the transformation matrix, which can then be applied to every subsequent frame.  

<div style="display: flex; justify-content: space-around; align-items: center;">

  <figure style="text-align: center;">
    <img src="vision_pictures/square_frame_contour.png" alt="Figure 1" style="width: 70%;"/>
    <figcaption>Figure 2: Corner squares detection</figcaption>
  </figure>

  <figure style="text-align: center;">
    <img src="vision_pictures/transformed_frame.png" alt="Figure 2" style="width: 70%;"/>
    <figcaption>Figure 3: Frame after perspective transformation</figcaption>
  </figure>

</div>

2. #### Detecting Global Obstacles  
Global obstacles are represented as black tiles on the map grid. To detect them, we examine each tile individually to determine whether it is predominantly black or white.  
Since this is a black-and-white classification problem, the frame is first converted to grayscale. A Gaussian blur is then applied to reduce noise, followed by a threshold to enhance contrast. We use a binary threshold, restricting values to 0 or 255, along with Otsu’s binarization algorithm. This ensures the threshold value adapts dynamically across the map, improving performance under varying lighting conditions.  
After preprocessing, each tile is masked using its centroid and edge size. The mean value of the masked area is calculated, and if the result is below a specified threshold (60 in our case), the tile is classified as black and thus identified as an obstacle.  

<div style="display: flex; justify-content: space-around; align-items: center;">

  <figure style="text-align: center;">
    <img src="vision_pictures/obstacles_blur.png" alt="Figure 1" style="width: 80%;"/>
    <figcaption>Figure 4: Frame after Gaussian Filter</figcaption>
  </figure>

  <figure style="text-align: center;">
    <img src="vision_pictures/obstacles_thresh.png" alt="Figure 2" style="width: 80%;"/>
    <figcaption>Figure 5: Frame after Otsu's binarization</figcaption>
  </figure>

</div>

3. #### Detecting the Goal  
The goal detection algorithm is similar to the map corner detection algorithm, with slight differences in preprocessing and color differentiation.  
For preprocessing, the image is converted to the HSV color space and filtered using a bilateral filter. This filter effectively reduces noise and grain while preserving edges.  
Instead of relying solely on the Hue value for color differentiation, we calculate the Euclidean distance using both Hue and Saturation values. Using only the Hue value can cause interference from black tiles, which may reflect similar Hue values to the goal reference color. However, the Saturation values differ significantly and help distinguish the goal from obstacles.  
Finally, the algorithm converts the detected goal position from frame coordinates to grid coordinates, allowing the path planning algorithm to interpret it accurately.

<div style="display: flex; justify-content: space-around; align-items: center;">

  <figure style="text-align: center;">
    <img src="vision_pictures/goal_hsv.png" alt="Figure 1" style="width: 120%;"/>
    <figcaption>Figure 6: Frame after conversion to HSV</figcaption>
  </figure>

  <figure style="text-align: center;">
    <img src="vision_pictures/goal_blur.png" alt="Figure 2" style="width: 120%;"/>
    <figcaption>Figure 7: Frame after bilateral filtering</figcaption>
  </figure>

  <figure style="text-align: center;">
    <img src="vision_pictures/goal_mask.png" alt="Figure 3" style="width: 120%;"/>
    <figcaption>Figure 8: Mask with goal highlighted</figcaption>
  </figure>

</div>

4. #### Robot Detection  
To detect and locate the robot, we use two colored dots on its top, aligned parallel to the wheel axis. In the direction of travel, the left dot is magenta, and the right dot is cyan.  
Before detection, the image is preprocessed. For circle detection, the frame is converted to grayscale as required by OpenCV’s *HoughCircles()* function. A median blur filter is applied to smooth the image, as it preserves edges better than Gaussian blur, which tends to reduce edge sharpness and complicate detection.  
The *HoughCircles()* function, based on a variant of the classic Hough Transform, is used for circle detection (refer to the code for parameter details). After detecting circles, we sort them by color. For each detected circle, we check its proximity to the reference colors and retain only the closest match for each reference color. We use the difference in Hue value. Finally, we ensure the centers of the two circles are not too far apart. We do not validate the circle radii, as this is already controlled by parameters in the Hough Transform function.  
With the centers of the two positioning dots identified, the robot's center is calculated as the midpoint between them. Knowing that the dots align with the wheel axis, we determine which is left and right and use trigonometry to compute the robot's heading angle. The position is then converted into millimeters for interpretation by the Kalman filter.  
Additionally, the algorithm outputs a tracking flag indicating whether the robot could be successfully tracked. This flag is used by other modules to determine whether the camera data can be relied upon.

<div style="display: flex; justify-content: space-around; align-items: center;">

  <figure style="text-align: center;">
    <img src="vision_pictures/pos_blur.png" alt="Figure 1" style="width: 80%;"/>
    <figcaption>Figure 9: Frame after Median Filter</figcaption>
  </figure>

  <figure style="text-align: center;">
    <img src="vision_pictures/pos_frame.png" alt="Figure 2" style="width: 80%;"/>
    <figcaption>Figure 10: Frame with robot position</figcaption>
  </figure>

</div>

### Additionnal information on BGR Vs HSV

At the beginning of the project, we used BGR values to detect and compare colors. However, this approach proved highly dependent on constant and uniform lighting across the map. To address this limitation, we switched to the HSV color space, which separates Hue, Saturation, and Value. This separation ensured a consistent Hue under varying lighting conditions, significantly improving the system's robustness to changes in illumination.


In [5]:
# Define a class for circles
class Circle:
    def __init__(self, center, radius, color):
        self.center = center  # Center position as a tuple (x, y)
        self.radius = radius  # Radius as a float
        self.color = color    # Color as a tuple (B, G, R)

    # Method to verify if the circle color matches a reference color
    def isColorMatching(self, ref_color):
        # Color distance
        ref_color = np.array(ref_color)
        dmax = 15 # max eculidian distance for the color to be considered equivalent
        d = distance(self.color[0], ref_color[0]) # only interested in the first HSV channel

        return d <= dmax

    def __repr__(self):
        return f"Circle(center={self.center}, radius={self.radius}, color={self.color})"
    

# Function to compute euclidian distance between two vectors
def distance(vec1, vec2):
    vec1 = np.array(vec1, dtype=np.float32)  # Ensure values are floats
    vec2 = np.array(vec2, dtype=np.float32)
    return np.linalg.norm(vec1 - vec2) # euclidian distance using NumPy

# Function to compute the median point between two circles
def circles_med(circle1, circle2):
    x1, y1 = circle1.center
    x2, y2 = circle2.center

    median_x = (x1 + x2)/2
    median_y = (y1 + y2)/2

    return (median_x, median_y)

# Function to compute the heading angle of the robot based on the two detection circles' centers
def robot_angle(cyan_center, mag_center):
    mag_center = np.array(mag_center, dtype=np.float32)  # Ensure values are floats
    cyan_center = np.array(cyan_center, dtype=np.float32)
    x_r, y_r = mag_center
    x_g, y_g = cyan_center
    
    # Calculate the direction vector
    dx = x_g - x_r
    dy = y_g - y_r
    
    # Compute the angle in radians
    angle_radians = np.arctan2(dy, dx)
    
    return angle_radians

# Funtion to detect the 4 corner squares of the map based on a reference color
def detect_squares(frame, ref_color):

    # Preprocessing
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) # Convert the frame to HSV values for better color separation
    blur = cv2.GaussianBlur(frame_hsv, (7, 7), 0) # Remove noise
    
    # Convert the frame to float for computation
    frame_float = blur.astype(np.float32)
    
    # Compute the Euclidean distance from the reference color
    ref_color = np.array(ref_color, dtype=np.float32)
    distance = np.abs(frame_float[..., 0] - ref_color[0]) # Only the first channel (Hue) is interesting
    
    # Threshold the image to isolate interest regions
    dmax = 10 # Maximum distance between the reference color and the pixel color
    mask = (distance <= dmax).astype(np.uint8) * 255 # Mask acting like a threshold

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    #Filter contours to detect squares
    squares = []
    for contour in contours:
        # Approximate the contour as a polygon
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Check if the contour is a square
        if len(approx) == 4 and cv2.isContourConvex(approx):
            area = cv2.contourArea(approx)
            if 300 < area < 1500:  # Filter by reasonable size
                squares.append(approx)

    # Ensure exactly 4 squares are detected
    if len(squares) != 4:
        print("Error: Expected 4 squares, but found", len(squares))
        return None
    else:
        print("Succes: 4 squares found !")

    # Compute squares centroids and edge length
    centroids = []
    for square in squares:
        # Calculate centroid
        centroid = np.mean(square.reshape(-1, 2), axis=0)
        
        # Calculate edge length (assuming square shape and uniform edges)
        edge_length = np.linalg.norm(square[0] - square[1])  # Distance between the first two vertices
        
        # Append centroid and edge length
        centroids.append(np.append(centroid, edge_length))
    
    centroids = np.array(centroids)

    return centroids


# Compute perspective transformation based on the centroids of the corner squares
def compute_transform(frame, centroids):
    # Sort centroids
    y_sort = centroids[centroids[:, 1].argsort()] # Sort based on the y-axis
    tops = y_sort[:2,:] # Two upper points
    tops = tops[tops[:,0].argsort()] # Sort based on the x-axis
    bottoms = y_sort[2:,:] # Two lower points
    bottoms = bottoms[bottoms[:,0].argsort()] # Sort based on the x-axis
    top_left = tops[0]
    top_right = tops[1]
    bottom_left = bottoms[0]
    bottoms_right = bottoms[1]

    # Shift the points from square centroids to map corners
    top_left = top_left[:2] - top_left[2]/2
    top_right = [top_right[0] + top_right[2]/2, top_right[1] - top_right[2]/2]
    bottom_left = [bottom_left[0] - bottom_left[2]/2, bottom_left[1] + bottom_left[2]/2]
    bottoms_right = bottoms_right[:2] + bottoms_right[2]/2

    # quadrilateral defining the map before transformation
    source_quad = np.array([top_left, top_right, bottom_left, bottoms_right], dtype=np.float32)

    # Define the destination rectangle
    rect_height, rect_width = frame.shape[:2]
    destination_rectangle = np.array([
        [0, 0],  # Top-left
        [rect_width, 0],  # Top-right
        [0, rect_height - 1],  # Bottom-left
        [rect_width - 1, rect_height - 1]  # Bottom-right
    ], dtype=np.float32)

    # Compute the perspective transform matrix
    matrix = cv2.getPerspectiveTransform(source_quad, destination_rectangle)

    return matrix

# Change of coordinates from frame referential to map referential
def frame2mapCoord(frame_coord):
    x, y = np.array(frame_coord)
    map_coord = np.array([-(y-427), x])
    return map_coord

# Change of coordinates from map referential to frame referential
def map2frameCoord(map_coord):
    x, y = np.array(map_coord)
    frame_coord = np.array([y, (-x)+427])
    return frame_coord

# Compute the centroids of every square of a grid
def grid_centroids(grid_width, grid_height, frame_width, frame_height):
    # Shape of the grid squares
    square_width = frame_width/grid_width
    square_height = frame_height/grid_height

    # Compute centroids
    centroids = np.empty((0, 2), dtype=int)
    for row in range(grid_height):
        for col in range(grid_width):
            x = round((col + 0.5) * square_width)
            y = round((row + 0.5) * square_height)
            centroids = np.vstack((centroids, [x, y]))
    
    return centroids

# Function detecting global obstacles on the map. Black squares are considered global obstacles
def detect_obstacles(frame, centroids, square_width, square_height):
    # Preprocessing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Convert to grayscale for black/white distinction
    blur = cv2.GaussianBlur(gray, (5,5),0) # Remove noise
    
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Detect black squares
    obstacles = np.empty((0, 2), dtype=int)
    for centroid in centroids:
        x, y = centroid
        
        # Create a blank mask
        mask = np.zeros(frame.shape[:2], dtype=np.uint8)

        # Define the square boundaries
        top_left = (int(x+square_width/2), int(y-square_height/2))
        bottom_right = (int(x-square_width/2), int(y+square_height/2))
        
        # Draw the region of interest on the mask
        cv2.rectangle(mask, top_left, bottom_right, 255, -1) # Black rectangle

        # Compute mean value of the region of interest
        mean = cv2.mean(thresh, mask=mask)[0]

        # Classify as obstacle or not
        if mean < 60:
            # Convert from pixel to grid position
            grid_x = round(x/square_width + 0.5)
            grid_y = round(y/square_height + 0.5)
            obstacles = np.vstack((obstacles, [grid_x, grid_y]))

    return obstacles

# Generate an array representing the map [0] are free squares and [-1] are obstacles.
# Converts a list of obstacle into a full-size map
def generate_map(grid_width, grid_height, obstacles):

    global_map = np.zeros((grid_height, grid_width))
    
    for obstacle in obstacles:

        # Change coordinate to align with the path generating function
        x, y = obstacle
        x_map = 6 - y
        y_map = x - 1
        global_map[x_map, y_map] = -1

    return np.transpose(global_map) # transpose to output a vertical map. Needed but the path generating function

# Function to detect the goal
def detect_goal(frame, square_width, square_height, ref_color):

    # Preprocessing
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) # Convert the frame to HSV values for better color separation
    blur = cv2.bilateralFilter(frame_hsv,15,75,150)

    # Bilateral Filter parameters
    # d=15: large filter region for better smoothing
    # sigmaColor=75: Default value to avoid too much color mixing
    # sigmaSpace=150: High value to filter noisy regions and preserv sharp edges
    
    # Convert the frame to float for precise computation
    frame_float = blur.astype(np.float32)
    # Compute the Euclidean distance from the reference color
    ref_color = np.array(ref_color, dtype=np.float32)
    distance = np.sqrt(np.sum((frame_float[..., :2] - ref_color[:2]) ** 2, axis=-1))

    # Threshold the image to isolate region of interest
    dmax = 40
    mask = (distance <= dmax).astype(np.uint8) * 255
    
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    #Filter contours to detect squares
    squares = []
    for contour in contours:
        # Approximate the contour
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Check if the contour is a square
        if len(approx) == 4 and cv2.isContourConvex(approx):
            area = cv2.contourArea(approx)
            if 100 < area < 1000:  # Filter by reasonable size
                squares.append(approx)

    # Ensure exactly 1 squares are detected
    if len(squares) != 1:
        print("Error: Expected 1 squares, but found", len(squares))
        return None
    else:
        print("Succes: 1 squares found !")

    # Compute goal centroid
    square = squares[0]
    goal_centroid = np.mean(square.reshape(-1, 2), axis=0)

    x, y = goal_centroid

    grid_x = round(x/square_width + 0.5)
    grid_y = round(y/square_height + 0.5)
    
    x_map = 6 - grid_y
    y_map = grid_x - 1

    goal_grid = np.array([x_map, y_map], dtype=np.int32)
    
    return goal_grid

In [6]:
def map_init():
    """
    Function initialising the map.
    """

    
    # Open Camera
    if not cap.isOpened():
        sys.exit("Failed to open camera. Exiting")
    print("Camera opened successfully.")

    # Dumping first frames to make sure the camera is calibrated
    for _ in range(100):
        cap.read()
    
    # Map detection and transformation
    for attempt in range(10):
        ret, frame = cap.read()
        if not ret:
            print(f"Warning: Frame capture failed on attempt {attempt + 1}.")
        else:
            # Detect centroids of the corner squares
            centroids = detect_squares(frame, MAG_REF_HSV)
            if centroids is not None and len(centroids)>0:
                print("Squares detected. Computing transformation matrix.")
                transformation_matrix = compute_transform(frame, centroids)
                return transformation_matrix
        print(f"Couldn't find the map :(, attempt {attempt + 1}")
    # If all attempts fail
    sys.exit("Couldn't find the map. Exiting")

async def cam_robot_pos(frame):
    # Reset per frames variables
    circles_detected = []
    cyan_circles = []
    mag_circles = []
    robot_position = None
    robot_angle_rad = None
    robot_position_map_coord = None
    tracking = False

    # Preprocessing for Hough transform
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(gray,7)

    # Compute circle Hough transform
    circles_hough = cv2.HoughCircles(
        blur, cv2.HOUGH_GRADIENT, dp=1, minDist=20,
        param1=150, param2=15, minRadius=5, maxRadius=15
    )

    # HoughCircles() parameter choice:
    # dp = 1: Keep the same resolution between input image and accumulator
    # minDist=20: Distance between centers is approx. 50mm = 25 pxl
    # param1=150: Higher threshold of the internal threshold. Value fixed experimentaly with cv2.Canny()
    # param2=15: low enough to detect all circles. False detected circles are not a big problem because we check color and distance
    # minRadius=5: radii approx. 17mm = 8.5 pxl
    # maxRadius=15: see above
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    if circles_hough is not None:
        circles_hough = np.uint16(np.around(circles_hough))       
        for i,circle in enumerate(circles_hough[0, :]):
            center = (circle[0], circle[1])
            radius = circle[2]
            # Mask for average color calculation
            mask = np.zeros(hsv.shape[:2], dtype=np.uint8)
            cv2.circle(mask, center, radius, 255, -1)
            avg_color = cv2.mean(hsv, mask=mask)[:3] #  alpha channel not useful
            circles_detected.append(Circle(center, radius, avg_color))
            
        if len(circles_detected) > 1:
            # Keep track of cyan and magenta
            for circle in circles_detected:
                # Check if the circle is cyan
                if circle.isColorMatching(CYAN_REF_HSV):
                    cyan_circles.append(circle)
                # Check if the circle is magenta
                elif circle.isColorMatching(MAG_REF_HSV):
                    mag_circles.append(circle)

            # Find the closest cyan and magenta circles to their references
            cyan_circle = min(cyan_circles, key=lambda c: distance(c.color, CYAN_REF_HSV), default=None)
            mag_circle = min(mag_circles, key=lambda c: distance(c.color, MAG_REF_HSV), default=None)

            # Validate and calculate robot position and orientation
            if cyan_circle and mag_circle:
                dist_between_circles = distance(cyan_circle.center, mag_circle.center)
                if dist_between_circles < MAX_CIRCLE_DISTANCE:
                    tracking = True

                    # Compute robot position and angle
                    robot_position = circles_med(cyan_circle, mag_circle)
                    robot_angle_rad = robot_angle(cyan_circle.center, mag_circle.center)

                    # Update trajectory
                    robot_position = tuple(map(int, robot_position))
                    robot_position_map_coord = frame2mapCoord(robot_position)

    if tracking:
        robot_position_map_mm = robot_position_map_coord*(1260/640) - [70 , 70]
        return robot_position_map_mm, robot_angle_rad, tracking
    else:
        print("Can't find the robot :(")
        return [-1,-1], -1, tracking

## 2. Global Navigation
The A* algorithm was selected to find the optimal path (in the sense of lowest cost) for the robot to travel along. The code for this algorithm was obtained from the solutions from week 5 and modified to adapt to our project specifications. 

In particular, the choice was made to implement the algorithm on a 9x6 grid, with displacement possible to all eight neighbors of a cell. However, movement to diagonal neighbors is only allowed if none of the neighbor cells which are adjacent to the diagonal neighbor are obstacles. In other words, the robot is not allowed to “cut corners” of an obstacle. If this constraint were not implemented, a part of the robot would pass over the obstacle during diagonal displacement. 

To take this into account, the diagonal neighbors cells were added to the neighbor list, and then a verification is made to check whether the adjacent neighbors are set as obstacles. In the positive case,  the corresponding diagonal neighbor is removed from the potential neighbor list. 

The g cost of moving to a diagonal was set to be equal to the one of regular adjacent displacement (1) multiplied by a factor root 2. For the heuristic function f, we chose the euclidean distance. This heuristic is slightly optimistic, because the actual path to the goal will always be larger or equal to the euclidean distance to the goal. So the heuristic does not overestimate the path length, therefore it is appropriate to use it.



In [7]:
## Code from exercises week 5, modified A* algorithm, BoMR course
def heuristic(a, b):
    # Implement the Euclidean distance heuristic
    return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def in_grid(grid, pos):
    return 0 <= pos[0] < grid.shape[0] and 0 <= pos[1] < grid.shape[1]

def is_obstacle(grid, pos):
    return not in_grid(grid, pos) or (grid[pos] == -1)

# A* algorithm implementation in a Grid
async def a_star_search(map_grid, start, goal):
    # Initialize the open set as a priority queue and add the start node
    open_set = []
    heappush(open_set, (heuristic(start, goal), 0, start))  # (f_cost, g_cost, position)

    # Initialize the came_from dictionary
    came_from = {}
    # Initialize g_costs dictionary with default value of infinity and set g_costs[start] = 0
    g_costs = {start: 0}
    # Initialize the explored set
    explored = set()
    operation_count = 0

    while open_set:
        # Pop the node with the lowest f_cost from the open set
        current_f_cost, current_g_cost, current_pos = heappop(open_set)

        # Add the current node to the explored set
        explored.add(current_pos)

        # For directly reconstruct path
        if current_pos == goal:
            break
        
        up         = (current_pos[0] - 1, current_pos[1]    )  # Up
        down       = (current_pos[0] + 1, current_pos[1]    )  # Down
        left       = (current_pos[0]    , current_pos[1] - 1)  # Left
        right      = (current_pos[0]    , current_pos[1] + 1)  # Right
        up_left    = (current_pos[0] - 1, current_pos[1] - 1)  # Up Left
        down_right = (current_pos[0] + 1, current_pos[1] + 1)  # Down Right
        down_left  = (current_pos[0] + 1, current_pos[1] - 1)  # Down Left
        up_right   = (current_pos[0] - 1, current_pos[1] + 1)  # Up Right
        # Get the neighbors of the current node (up, down, left, right)
        neighbors = [up, down, left, right, up_left, down_right, down_left, up_right]
        
        # exclude diagonal displacements if there is an adjacent obstacle (no corner cutting)
        if is_obstacle(map_grid, up) or is_obstacle(map_grid, left) :
                neighbors.remove(up_left)
        
        if is_obstacle(map_grid, up) or is_obstacle(map_grid, right):
                neighbors.remove(up_right)
                
        if is_obstacle(map_grid, down) or is_obstacle(map_grid, right):
                neighbors.remove(down_right)
                
        if is_obstacle(map_grid, down) or is_obstacle(map_grid, left):
                neighbors.remove(down_left)
            
                
        for neighbor in neighbors:
            # Check if neighbor is within bounds and not an obstacle
            if (in_grid(map_grid,neighbor)) and map_grid[neighbor[0], neighbor[1]] != -1 and neighbor not in explored:
                # Calculate tentative_g_cost
                tentative_g_cost = current_g_cost + heuristic(current_pos, neighbor)
                
                # If this path to neighbor is better than any previous one
                if neighbor not in g_costs or tentative_g_cost < g_costs[neighbor]:
                    # Update came_from, g_costs, and f_cost
                    came_from[neighbor] = current_pos
                    g_costs[neighbor] = tentative_g_cost
                    f_cost = tentative_g_cost + heuristic(neighbor, goal)

                    # Add neighbor to open set
                    heappush(open_set, (f_cost, tentative_g_cost, neighbor))
                    operation_count += 1

    # Reconstruct path
    if current_pos == goal:
        path = []
        while current_pos in came_from:
            path.append(current_pos)
            current_pos = came_from[current_pos]
        path.append(start)
        return path[::-1], explored, operation_count
    else:
        # If we reach here, no path was found
        return None, explored, operation_count

With the most efficient path stored, the critical points, meaning changes in orientation, were stored in a new list that defined the goals for the controller. This list is produced by the following function.

**After testing, we figured out that have all the intermediate goals improves tracking of the path. This function is not used further in the code. We kept it in the report for documentation purposes.**

In [8]:
# Takes the path found# To keep track of how many redundant steps of the path have been removed  by the A* algorithm as an argument and removes the points with a repeated orientation

async def goal_list_from_path(path):
    goal_list = path.copy()
    popped = 0    # To keep track of how many redundant steps of the path have been removed 
    for i in range(len(path)-2):
        # Checks if three consecutive points lie in a straiExght line and removes middle because it is redundant
        if (path[i+1][0] - path[i][0] == path[i+2][0] - path[i+1][0]) and (path[i+1][1] - path[i][1] == path[i+2][1] - path[i+1][1]):
            goal_list.pop(i+1 - popped)
            popped += 1
    return goal_list

## 3. Local Navigation
The local navigation relies on the horizontal sensors of the Thymio. If an object is detected, the robot changes the velocity of the motors to effectively avoid it. The motor velocity changes are made through the weights. Each weight determines how a sensor impacts the robot's movement which is why the weights are larger for the exterior sensors pushing the robot to turn more aggressively than the center weights. The weights were chosen based on the code provided in the third week exercises on Artificial neural networks and adapted according to the behavior desired, through trial and error. The final weights were chosen because they provided a behavior that was not too aggressive, meaning the motor velocities were not too high, yet the obstacle was appropriately avoided.  

In addition to the set weights, an additional component was added to ensure the aggressive nature of the local avoidance response was diminished. Specifically, if the robot senses the object getting further away, the weight influence is decreased by a factor of 2. This guarantees that as the robot moves further away from the object, the robot's avoidance response becomes more measured and less aggressive.

The code can be seen as a single-layer neural network where the input is the proximity sensor (7 elements) and the output is each motor (2). Parts of the code were taken from the solutions to the third week exercises on Artificial Neural Networks. It is important to note that although the seven sensors were implemented, a circumstance in which having an obstacle behind the Thymio did not arise since the Thymio always moves forward.  

**Local Obstacle Positioning conditions**

In order for the robot to not collide with global obstacles while in local avoidance state, the local obstacle must be placed in a sufficiently conservative position which does not prompt the robot to fail in a near future. The meaning of sufficient is up to the operator.

In [9]:
## Function to obtain the sensor values from the selected type of sensors. Similar to function print_sensor_values obtained from solutions for week 3 exercises

async def get_sensor_values(sensor_id, delay=0.01):
    await node.wait_for_variables({str(sensor_id)})
    await client.sleep(delay)
    sensor_values = list(node[sensor_id])  # Retrieve the current values
    return sensor_values  # Return the collected sensor values as a list

In [10]:
## Function to verify if there are any obstacles. Gets the current readings of the get_sensor_values function for the horizontal sensors
## If the value of the sensor is above the proximity threshold, it sends a 1

async def check_obstacles(proximity_threshold=1500):
    horizontal_sensor_values = await get_sensor_values('prox.horizontal')
    if any(x > proximity_threshold for x in horizontal_sensor_values):
        return 1  # Obstacle detected
    return 0  # No obstacle

In [ ]:
## Function to avoid obstacles based on neural network
## Obtained from solutions for week 3 exercises and adapted to satisfy speed requirements and asynchronous functionality

async def obstacle_avoidance(left_output=0, right_output=0, mean_prev=0):
    # Define weights for left and right motors
    w_l = [ 25,   15, -10, -20, -30,  0, 0]
    w_r = [-30, -20, -10,    15, 25, 0,  0]

    # Scale factors for sensors and constant factor
    sensor_scale = 2000
    
    # Read proximity sensor values
    prox_values = list(node['prox.horizontal'])

    mean_values = np.mean(prox_values)

    if mean_values < mean_prev:
        sensor_scale *= 2
    
    # Compute motor outputs based on the scaled sensor values
    for i in range(len(prox_values)):
        # Scale the sensor readings
        scaled_value = prox_values[i] // sensor_scale
        
        # Update outputs for left and right motors
        left_output += scaled_value * w_l[i]
        right_output += scaled_value * w_r[i]

    # Set motor powers
    await node.set_variables(motors(int(left_output), int(right_output)))
    
    return left_output, right_output, mean_values

## 4. Kalman Filter
The Kalman Filter’s state representation encapsulates the robot’s position, orientation, linear velocities, and angular velocity, the necessary parameters required for accurate navigation and control in a 2D environment. The state transition model was designed as a linear discrete-time state-space model for each of the components, as in the 8th practice session, assuming constant velocities over constant discrete time intervals (Ts). The time interval was chosen based on the time step delta_t = 0.1s defined in the main control loop and also on the duration of execution of the code, which slightly increases the time between two iterations of the loop. In fact, initially this additional time was not taken into account in the Kalman filter. This led to the estimate of the Kalman filter displaying a non negligible lag behind the true physical position of the robot when the camera was covered and the robot could not be tracked visually. The problem was successfully  identified, so a measure of the duration of execution of the control loop was made with the time library in python. On average, this execution took about 0.02 seconds. So the time constant of the Kalman filter was set to Ts = delta_t + 0.02, that is 0.12 seconds. 

Two matrices were used to model the uncertainties of the robot motion model, the process noise covariance matrix (Q) and the measurement noise covariance matrix (R). For simplicity, it is assumed that the measurement noise on position and speed are independent.  Similarly, for the noise on the position in x and y, which depends on the angle, the maximum value is taken at all times.

*For the Q matrix:*
- Position noise (q_x and q_y = 18 mm$^2$): these values were chosen by assuming there are about 3 pixels of error from the camera, which corresponds to  6 mm. Therefore the noise variance is 6^2 = 36 mm$^2$. Assuming half of the noise comes from the process, and half from the measurement, q_p = 36/2 = 18 mm$^2$. To ensure robustness, the worst-case scenario values are consistently used, q_x = 18 mm$^2$ and q_y = 18 mm$^2$.
- Orientation noise (q_phi = 0.0025 rad$^2$): It was estimated that the total orientation noise is around 5°, that is, 0.05 rad. Therefore, the orientation variance is 0.0025 rad$^2$. Assuming half comes from the process, and half from the measurement as before, the orientation noise is 0.0025/2 = 0.00125 rad$^2$.
- Velocity noise (q_vx and q_vy = 0.74 mm$^2$ /$s^2$): To obtain the velocity noise, the robot was given a constant speed of 119 thymio unit (corresponds to 4cm/s, which is our working speed) on both wheels for 25 seconds. Each 0.1 s, the speed was measured and converted to mm/s using the speed conversion factor. After all the data was collected, the variance was computed resulting in a value of 1.48 mm$^2$/ $s^2$. Taking half of this for the process noise, gives q_vx and q_vy = 0.74 mm$^2$ /$s^2$.
- Angular velocity noise (q_vphi = 0.0000779 rad$^2$ /$s^2$) was measured with the same procedure as the velocity noise, however, it was computed based on the measured changes in orientation during the experiment. 

*For the R matrix:*
- Camera measurements for position (r_x and r_y = 18 mm$^2$): Given that is was assumed that half of the noise variance came from the measurement, r_x and r_y are identical to q_x and q_y, that is r_x = 18 mm$^2$ and r_y = 18 mm$^2$.
- Camera’s angular measurement noise (r_phi = 0.0025 rad$^2$ ): Following the same reasoning as above, the camera’s angular measurement noise was set to 0.00125 rad$^2$ in the Kalman filter. 
- Velocity measurement noise (r_vx and r_vy = 0.74 mm$^2$ /$s^2$): Similar as the velocity noise variance, half of the measured value, 1.48 mm$^2$/s, was attributed to the measurement noise. 
- Angular velocity noise (r_vphi = 0.0000779 rad$^2$ /s$^2$) was derived experimentally, similar to linear velocity noise, by measuring the robot’s rotational motion, then split equally, resulting in a value of 0.4. 


*Speed conversion factor calculation:* The speed conversion factor was computed by making the Thymio go forward at a set speed (e.g., 50,50 or 100,100) for a set amount of time (e.g., 3 seconds or 6 seconds) and then measuring the physical distance traveled by the robot. The average speed of the robot was then computed by dividing the distance measurement by the time set in the program. This was done for many combinations of speeds and durations, and then the empirical mean of the ratio of average speed to Thymio motor speed was computed. Said ratio corresponds to the speed conversion factor. The factor was found to be approximately 0.33615 mm/s.

*Angular speed conversion factor calculation:* In a similar manner, the angular speed conversion factor was found to be 0.003356 rad/s. 


The Kalman Filter operates in two stages: prediction and correction. In the prediction stage, the robot’s state is estimated based on the motion model, using matrix A and the previous state. This provides an a priori estimate, incorporating process noise from Q. During the correction stage, sensor data is used to refine the predictions.

Within the code, two distinct cases exist:
1) The camera is operational and it is possible to estimate the different speeds with the motors and the position with the camera.
2) The camera is not operational, thus, it is only possible to estimate the different speeds with the motors.

To know which case should be employed, the camera sends a flag to indicate whether the camera can see the Thymio or not. As a result, robot movement is possible without the camera, even if the position error increases. This operation requires there is an H matrix and a y vector depending on the case we are in.

*Critical angle problem*

Since the angles are measured from $-\pi$ to $\pi$, there was a problem when the angle estimated by the Kalman filter was close to $pi$, and the one found by the computer vision was close to $-pi$ (or vice-versa). While the physical position of the robot and the estimation were very close, the numerical difference between these two angles was very large, so the kalman filter's estimation would induce a very large correction, generating erratic behavior of the robot. To solve this, some conditions were added to adjust the angle difference in these edge cases.

In [12]:
# Initialising the remaining constants
# units: x [mm],
#        y [mm],
#        phi [°],
#        v_x [mm/s]
#        v_y [mm/s]
#        phi_point [°/s]
#        Ts [s]


Ts = 0.12

# Matrix that creates model 
A = np.array([[1, 0, 0, Ts, 0, 0],
              [0, 1, 0, 0, Ts, 0],
              [0, 0, 1, 0, 0, Ts],
              [0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 1]])

# Variables for Q matrix to control the variance as a function of the sensors
q_x = 18 # mm2
q_y = 18 # mm2
q_phi = 0.00125 # rad2
q_vx = 0.74 # mm2/s2
q_vy = 0.74 # mm2/s2
q_vphi = 0.000079 # rad2/s2

Q = np.array([[q_x, 0, 0, 0, 0, 0],
              [0, q_y, 0, 0, 0, 0],
              [0, 0, q_phi, 0, 0, 0],
              [0, 0, 0, q_vx, 0, 0],
              [0, 0, 0, 0, q_vy, 0],
              [0, 0, 0, 0, 0, q_vphi]])

r_x = 18 # mm2
r_y = 18 # mm2
r_phi = 0.00125 # rad2
r_vx = 0.74 # mm2/s2
r_vy = 0.74 # mm2/s2
r_vphi = 0.000079 # rad2/s2

speed_conv_factor = 0.33615; #valeur moteur -> mm/s
angular_speed_conv_factor = 0.1923*np.pi/180 #delta valeur moteur -> rad/s

async def kalman_filter(speed_left, speed_right, camera_x, camera_y, camera_phi, camera_flag, x_est_prev, P_est_prev,
                  HT=None, HNT=None, RT=None, RNT=None):
    """
    Estimates the current state using input sensor data and the previous state
    
    param speed_left: measured speed of motor left(Thymio units)
    param speed_right: measured speed of motor right(Thymio units)
    param camera_x: measured x coordinate with the camera
    param camera_y: measured y coordinate with the camera
    param camera_phi: measured phi angle with the camera
    param camera_flag: flag to know if the camera is operational 
    param x_est_prev: previous state a posteriori estimation
    param P_est_prev: previous state a posteriori covariance
    
    return x_est: new a posteriori state estimation
    return P_est: new a posteriori state covariance
    """
    
    ## Prediciton through the a priori estimate
    # estimated mean of the state
    x_est_a_priori = np.dot(A, x_est_prev);
    
    # Estimated covariance of the state
    P_est_a_priori = np.dot(A, np.dot(P_est_prev, A.T));
    P_est_a_priori = P_est_a_priori + Q if type(Q) != type(None) else P_est_a_priori

    # critical angle condition 1
    if (x_est_a_priori[2][0] - camera_phi) > np.pi:
        camera_phi += 2 * np.pi

    # critical angle condition 2
    if (x_est_a_priori[2][0] - camera_phi) < - np.pi:
        camera_phi -= 2 * np.pi
    
    #speed 
    speed = (speed_left + speed_right)/2
    speed_angular = (speed_left - speed_right) #si positif -> sens horaire 

    phi = x_est_a_priori[2][0]

    ## Update         
    # y, H, and R for a posteriori estimate, depending on transition
    if (camera_flag): 
        
        y = np.array([[camera_x],
                      [camera_y],
                      [camera_phi],
                      [speed*speed_conv_factor*np.cos(phi)],
                      [speed *speed_conv_factor*np.sin(phi)],
                      [speed_angular*angular_speed_conv_factor]])
        H = np.array([[1, 0, 0, 0, 0, 0],
                      [0, 1, 0, 0, 0, 0],
                      [0, 0, 1, 0, 0, 0],
                      [0, 0, 0, 1, 0, 0],
                      [0, 0, 0, 0, 1, 0],
                      [0, 0, 0, 0, 0, 1]])
        R = np.array([[r_x, 0, 0, 0, 0, 0],
                      [0, r_y, 0, 0, 0, 0],
                      [0, 0, r_phi, 0, 0, 0],
                      [0, 0, 0, r_vx, 0, 0],
                      [0, 0, 0, 0, r_vy, 0],
                      [0, 0, 0, 0, 0, r_vphi]])
    else:
        # no transition, use only the speed
        y = np.array([[speed*speed_conv_factor*np.cos(phi)],
                      [speed *speed_conv_factor*np.sin(phi)],
                      [speed_angular*angular_speed_conv_factor]])
        H = np.array([[0, 0, 0, 1, 0, 0],
                      [0, 0, 0, 0, 1, 0],
                      [0, 0, 0, 0, 0, 1]])
        R = np.array([[r_vx, 0, 0],
                      [0, r_vy, 0],
                      [0, 0, r_vphi]])

    # innovation / measurement residual
    i = y - np.dot(H, x_est_a_priori);
    # measurement prediction covariance
    S = np.dot(H, np.dot(P_est_a_priori, H.T)) + R;
             
    # Kalman gain (tells how much the predictions should be corrected based on the measurements)
    K = np.dot(P_est_a_priori, np.dot(H.T, np.linalg.inv(S)));
    
    
    # a posteriori estimate
    x_est = x_est_a_priori + np.dot(K,i);
    P_est = P_est_a_priori - np.dot(K,np.dot(H, P_est_a_priori));

    # make sure the angle stays between pi and -pi
    if (x_est[2][0]) > np.pi:
        x_est[2][0] -= 2 * np.pi

    if (x_est[2][0]) < - np.pi:
        x_est[2][0] += 2 * np.pi
     
    return x_est, P_est

## 5. Motion Control

### 5.1 PID Control

To effectively create a controller to ensure the robot is able to follow the path, a PID controller based on the orientation error of the robot was employed. A measure of the orientation is obtained through the camera and processed with the Kalman filter to control the robot. A series of functions were used to create the controller based on the error measured between the desired orientation with respect to a certain goal position and the orientation estimate from the Kalman filter. The appropriate gains were selected after testing their effect and identifying the desired response for the robot. The following considerations were taken into account:

#### Proportional Gain $K_p$  <!--  (\(K_p\))  -->
- Controls the strength of the robot's response to the current error.
- Determines the time taken to correct the error (avoiding overshooting or overly slow responses).

#### Integral Gain  $K_i$ <!--  (\(K_i\))  -->
- Controls the accumulation of errors in the past through an integration operation.
- Ensures there are no steady-state errors by compensating for persistent offsets.

#### Derivative Gain  $K_d$ <!--  (\(K_d\))  -->
- Controls the robot's reaction to the rate of change of the error.
- Determines the sensitivity of the controller, ensuring the correction rate is neither too aggressive nor too delayed.

To test the controller, a simulator was employed, and the corresponding codes are also included. In the end, we set $K_i$ to 0, which means we only make use of a PD controller.

The following function takes as input the desired x and y positions as well as the current positions estimate (from the kalman filter) to get the errors. This will be used in the PID controller. For position, the Euclidean distance norm is used: 
\begin{equation} 
dist error = \sqrt{x_{err}^2 + y_{err}^2} 
\end{equation}

For orientation, the following equation is used:
\begin{equation} 
\phi_d = \arctan2(y_{\text{err}}, x_{\text{err}}) 
\end{equation} 
\begin{equation} 
\phi_{\text{err}} = \phi_d - \phi 
\end{equation}

The function $\arctan2(y,x)$ returns a value in the interval $(-\pi,\pi]$ for the angle $\theta$ of a vector $(x,y)$, with respect to the $x$ axis.

Finally, the orientation error is then normalized to the range -180° to 180° (in case the signs of desired and estimated orientations are different and the sum of their norm is greater than $\pi$).

In [13]:
## This function calculates the position and orientation errors based on the estimated and desired positions of the robot
# from https://felipenmartins.github.io/Robotics-Simulation-Labs/Lab4/

async def get_pose_error(xd, yd, x, y, phi):
    """ Returns the position and orientation errors. 
        Orientation error is bounded between -pi and +pi radians.
        xd: deisred x position
        yd: desired y position
        x:  estimated x position
        y:  estimated y position
        phi: estimated orientation
    """
    # Position error:
    x_err = xd - x 
    y_err = yd - y
    dist_err = np.sqrt(x_err**2 + y_err**2) # Euclidean distance

    # Orientation error
    phi_d = np.arctan2(y_err,x_err) # The desired orientation phi_d is computed online from the vector of position error
    phi_err = phi_d - phi

    # Limits the error to (-pi, pi):
    phi_err_correct = np.arctan2(np.sin(phi_err),np.cos(phi_err))

    return dist_err, phi_err_correct

This definition defines a PID controller defining the gains for each of the terms. It takes as input the current error, the error from the previous step, the accumulated error and the time step and applies the following equation. The Gains can be declared as a result of trial and error. The PID function is composed of the following:
\begin{equation} P = k_p \cdot e \end{equation}
\begin{equation} I = e_{\text{acc}} + k_i \cdot e \cdot \Delta t \end{equation}
\begin{equation} D = k_d \cdot \frac{e - e_{\text{prev}}}{\Delta t} \end{equation}
\begin{equation}
\text{Output} = k_p \cdot e + k_i \cdot \int e \, dt + k_d \cdot \frac{de}{dt}
\end{equation}

In [14]:
## This function implements a "goal to goal" behavior meaning that it controls the robot orientation by adjusting the angular speed only. 
    # The forward speed is supposed to remain constant, equal to a chosen value
# from https://felipenmartins.github.io/Robotics-Simulation-Labs/Lab4/

async def pid_controller(e, e_prev, e_acc, delta_t, kp=1.0, kd=0, ki=0):
    """ 
    PID algortithm: must be executed every delta_t seconds
    The error e must be calculated as: e = desired_value - actual_value
    e_prev contains the error calculated in the previous step.
    e_acc contains the integration (accumulation) term.
    """
    P = kp*e                      # Proportional term; kp is the proportional gain
    I = e_acc + ki*e*delta_t      # Intergral term; ki is the integral gain
    D = kd*(e - e_prev)/delta_t   # Derivative term; kd is the derivative gain

    output = P + I + D              # controller output, angular velocity in this case

    # store values for the next iteration
    e_prev = e     # error value in the previous interation (to calculate the derivative term)
    e_acc = I      # accumulated error value (to calculate the integral term)

    return output, e_prev, e_acc

Taking the output of the controller (angular speed), the speed for each wheel is calculated with the following equations, which relate the forward speed $u_d$ and the angular speed $\omega_d$ of a differential robot to the angular speeds of each of the wheels (left $\omega_{l_d}$ and right $\omega_{r_d}$). The linear speed remains constant to avoid complex behaviors and inaccurate position estimation. $r$ is the radius of the robot’s wheels, $d$ is the base width, that is, the distance between the center of the two wheels.
\begin{equation} \omega_{r_d} = \frac{2 \cdot u_d - d \cdot \omega_d}{2 \cdot r} \end{equation} 
\begin{equation} \omega_{l_d} = \frac{2 \cdot u_d + d \cdot \omega_d}{2 \cdot r} \end{equation}

In [15]:
## This function calculates the speeds for the left and right wheels based on the desired values of linear and angular speeds
    # The equations come from a model of the kinematics of a differential robot. 

# modified from https://felipenmartins.github.io/Robotics-Simulation-Labs/Lab4/
    # since we chose a different reference frame, we had to invert the wr_d and wl_d from the original code.

async def wheel_speed_commands(u_d, w_d, d, r, phi_erreur):
    """
    Convert desired robot speeds to desired wheel abgular speeds
    """
    wr_d = float((2 * u_d - d * w_d) / (2 * r))
    wl_d = float((2 * u_d + d * w_d) / (2 * r))
        
    return wl_d, wr_d

In [16]:
async def check_kidnapping():
    # Get the current ground sensor values
    ground_sensor_values = await get_sensor_values('prox.ground.reflected')

    # Threshold to detect lifting
    ground_threshold = 25  

    # Check if both ground sensors are below the threshold
    if ground_sensor_values[1] < ground_threshold and ground_sensor_values[0] < ground_threshold:
        print("Kidnapping detected! Robot lifted off the ground. Check_kidnapping function")
        return 1  # Return 1 for kidnapping state
    else:
        return 0  # Normal operation

### 5.2 Main Control Loop combining the global and local navigation


The main control loop is the basis of the robot's navigation system, allowing the robot to follow the global path created with A* while continuously checking for local obstacles and adjusting its movement using the estimates of the kalman filter and the commands of the PD controller. The loop is composed of two while loops to ensure that the code continues to run until the target is reached. The main while loop represents a finite state machine. As seen in the graphic, it is composed of 3 states: global path following (state 0), obstacle avoidance (state 1), and kidnapping (state 2). If no obstacle is detected and the robot is not kidnapped, only state 10 (initial) and state 0 are observed. The state 10 includes the code that needs to be executed each time the robot changes position or is initialized. It was added as part of the exterior while loop to ensure that even if the robot is kidnapped, the code continues to run. When it is set on the ground it recomputes the position and the global path to reenter the finite state machine until the target is reached. 

If at any point the robot detects a local obstacle, it enters state 1 where the code to follow the global path is overwritten and the robot relies solely on sensor inputs to function. When entering state 1, a single intermediary goal is skipped in the path in order to avoid having the Thymio loop backward when exiting the local obstacle avoidance state. When an obstacle is no longer detected, it continues its path towards the next point in the global navigation trajectory. When returning to the path following state from local obstacle avoidance, to make the transition to the next goal point smoother, the value of the proportional gain is diminished for a defined number of iterations. This diminishes the chances of the robot encountering the same obstacle again. It is important to note that the reduction ratio of the proportional gain is not constant during this period, rather it decreases linearly with time (i.e. each iteration). Its lowest value is 1 after 1.5 seconds, at which point the proportional gain has returned to its original value.

Similarly, if the robot no longer detects the ground, the motors are stopped and the finite state machine is exited. The robot returns to state 10, and does not recalculate its new position and trajectory until it is placed on the ground. 

<div style="display: flex; justify-content: space-around; align-items: center;">

  <figure style="text-align: center;">
    <img src="img/main_loop.jpg" alt="Figure 6" style="width: 100%;"/>
    <figcaption>Figure 11</figcaption>
  </figure>

</div>


In [17]:
## Function from exercises week 8, BoMR course

def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

In [18]:
## For testing functionality of the motor (signal reception)
node.send_set_variables(motors(100, 100))

In [19]:
## For testing functionality
node.send_set_variables(motors(0, 0))

The main control loop defines several initial values which were established based on testing results. 

In [20]:
async def main_control_loop(transformation_matrix):
    """
    Main control loop structured as a finite state machine.
    """

    # Initialize positions arrays
    cam_positions = np.empty((0, 2), dtype=int)
    kalman_positions = np.empty((0, 2), dtype=int)
    
    # Starting state
    state = 10
    
    # Constant forward speed
    u_const = 4  # cm/s

    # Grid size
    grid_width = 9
    grid_height = 6

    # Get frame for initial obstacle detection and robot position
    for i in range(5):
        ret, frame = cap.read()
        if not ret:
            print(f"Camera failed for initial obstacle detection and robot position at attempt {i + 1}")
        else:
            break
    frame_height, frame_width = frame.shape[:2]
    frame = cv2.warpPerspective(frame, transformation_matrix, (frame_width, frame_height))
    frame = cv2.resize(frame, (640, 427)) # Resizing with 3/2 aspect ratio
    frame_height, frame_width = frame.shape[:2] # Update frame shape
    
    # Map
    centroids = grid_centroids(grid_width, grid_height, frame_width, frame_height)
    square_width = frame_width/grid_width
    square_height = frame_height/grid_height

    obstacles = detect_obstacles(frame, centroids, square_width, square_height)
    Map = generate_map(grid_width, grid_height, obstacles)

    # Robot geometric parameters
    base_width   = 9.4   # cm
    wheel_radius = 2.2 # cm

    # Speed constant (convert thymio motor speed to forward speed)
    speed_constant = 0.03375 # cm/s

    # Time step
    delta_t = 0.1  # seconds

    # Initial errors for PID
    phi_err_prev = 0
    phi_err_acc = 0

    # Set initial wheel speed
    speed_left = 0
    speed_right = 0
    
    # Signal to finish running the code indefinetely
    done = False

    # Set initial wheel speed
    speed_left = 0
    speed_right = 0

    # Local obstacle timer

    timer = 0 
    mean_prev = 0
    local_obstacle_flag = 0

    P_est = 1000 * np.ones(6)

    while not done:
        
        if state == 10:
            cam_positions = np.empty((0, 2), dtype=int)
            kalman_positions = np.empty((0, 2), dtype=int)
            
            node.send_set_variables(motors(0, 0))
            #time.sleep(3) # This can be removed, only used to adjust robot at the beginning and put back from kidnapping

            # While loop used to detect whether the robot is on the ground, if it is not, it will be in the loop 
            threshold = 400
            while True:
                # Get ground sensor values
                ground_sensor_values = await get_sensor_values('prox.ground.reflected')
                
                if ground_sensor_values[1] >= threshold and ground_sensor_values[0] >= threshold:
                    print("Robot on the ground :)")
                    break  # if the robot is placed on the ground

                # Black frame for kidnapped scenario
                kidnapped_frame = np.zeros((frame_height, frame_width), dtype=np.uint8)
                
                # Define the text and font properties
                text = "Kidnapped..."
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 2
                font_thickness = 3
                text_color = 255  # White color
                
                # Get the text size
                text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
                text_width, text_height = text_size
                
                # Calculate the text position to center it
                text_x = (frame_width - text_width) // 2
                text_y = (frame_height + text_height) // 2  # Add text_height to center vertically
                
                # Split text into lines and draw each line
                cv2.putText(kidnapped_frame, text, (text_x, text_y), font, font_scale, text_color, font_thickness)

                # Display the frame
                cv2.imshow(win_name, kidnapped_frame)
                cv2.waitKey(1)
            
                print('I am being kidnapped :(, put me on the ground to continue')
                time.sleep(delta_t) 

            time.sleep(3) # So the person can move and the camera can detect the position
            
            # Goal detection after kidnapping
            for attempt in range(10):
                ret, frame = cap.read()
                if not ret:
                    print(f"Camera failed for obstacle detection and robot position after kidnapping at attempt {attempt + 1}")
                else:
                    frame_height, frame_width = frame.shape[:2]
                    frame = cv2.warpPerspective(frame, transformation_matrix, (frame_width, frame_height))
                    frame = cv2.resize(frame, (640, 427)) # Resizing with 3/2 aspect ratio
                    
                    SearchGoal = detect_goal(frame, square_width, square_height, CYAN_REF_HSV)
                    if SearchGoal is not None:
                        print("Goal found after kidnapping !", SearchGoal)
                        break
                    print(f"Couldn't find the goal after kidnapping :(, attempt {attempt + 1}")
            SearchGoal = [SearchGoal[1], SearchGoal[0]]

            # Robot detection after kidnapping
            for attempt in range(10):
                ret, frame = cap.read()
                if not ret:
                    print(f"Camera failed for obstacle detection and robot position after kidnapping at attempt {attempt + 1}")
                else:
                    frame_height, frame_width = frame.shape[:2]
                    frame = cv2.warpPerspective(frame, transformation_matrix, (frame_width, frame_height))
                    frame = cv2.resize(frame, (640, 427)) # Resizing with 3/2 aspect ratio
                    
                    SearchStart, StartAngle, tracking = await cam_robot_pos(frame)
                    if tracking:
                        SearchStart_pxl = map2frameCoord(((SearchStart + [70, 70])*(640/1260)).astype(int)) # Convert to pixels
                        cam_positions = np.vstack((cam_positions, SearchStart_pxl))
                        print("Robot found after kidnapping !", SearchStart)
                        break
                print(f"Couldn't find the robot after kidnapping :(, attempt {attempt + 1}")
            XStart = SearchStart[0]
            YStart = SearchStart[1]
            SearchStart = np.round(np.array([SearchStart[1]/140, SearchStart[0]/140])).astype(int)
            
            # Obtaining path
            print('Obtaining path')
            path, explored, operation_count = await a_star_search(Map, tuple(SearchStart), tuple(SearchGoal))
            if path is None:
                sys.exit("No valid path found")
            # Extract the key_points
            goal_list = path

            # Kalman initialisation
            x_est = np.array([[XStart],
                              [YStart],
                              [StartAngle],
                              [0],
                              [0],
                              [0]])
        
            # Start with the first goal
            next_goal = 1
            x_goal = goal_list[next_goal][1]
            y_goal = goal_list[next_goal][0]
        
            it = 0
    
            state = 0
    
        while next_goal < len(goal_list):
    
            # Check if there are any obstacles
            bool_obstacles = await check_obstacles(proximity_threshold=2000)
            
            # Check to see if robot position changed very suddenly & ground sensors detected lifting
            bool_kidnap = await check_kidnapping()

            # Combining states
            if bool_kidnap == 1:
                state = 2  # Kidnapping 
            elif bool_obstacles == 1:
                state = 1  # Obstacle 
                
            else:
                state = 0  # Normal operation
            
            if state == 0:
                it += 1

                x = x_est[0][0]/140
                y = x_est[1][0]/140
                phi = x_est[2][0]

                # Calculate pose error, only used orientation because the distance error is not necessary
                _, phi_err = await get_pose_error(x_goal, y_goal, x, y, phi) # the first returned value is the error in position, but we do not use this information 
                
                # Update orientation PID
                k_p = 0.9
                if timer > 0:
                    timer -= 1
                    k_p /= ((timer+4)/4)
                    
                
                phi_control, phi_err_prev, phi_err_acc = await pid_controller(
                    phi_err, phi_err_prev, phi_err_acc, delta_t, kp=k_p, kd=0.05, ki=0.0)
    
                # Map control outputs to desired forward and angular speeds
                u_d = u_const  # Constant forward speed
                w_d = phi_control  # Angular speed from PID
    
                # Compute wheel speed commands
                left_angular, right_angular = await wheel_speed_commands(u_d, w_d, base_width, wheel_radius, phi_err) # returns angular velocity for each wheel
                
                # Scale to motor commands
                left_speed  = left_angular  * wheel_radius / speed_constant
                right_speed = right_angular * wheel_radius / speed_constant
    
                node.send_set_variables(motors(int(left_speed), int(right_speed)))
    
                # Check if goal is reached
                dist_to_goal = np.sqrt((x_goal - x)**2 + (y_goal - y)**2)*14
                if dist_to_goal < 3:  # 3 cm tolerance
                    local_obstacle_flag = 0
                    print('******************************* goal reached *******************************')
                    next_goal += 1
                    print(f"Before loop check, next_goal = {next_goal}, len(goal_list) = {len(goal_list)}")
                    if next_goal < len(goal_list):
                        x_goal = goal_list[next_goal][1]
                        y_goal = goal_list[next_goal][0]
                    elif next_goal == len(goal_list):
                        print('Goal Reached!')
                        done = True # Flag to terminate outer loop
                        break
    
                # Wait for the next iteration (optional for simulation)
            elif state == 1:
                if local_obstacle_flag == 0 and next_goal < len(goal_list) - 1:
                    next_goal += 1
                    x_goal = goal_list[next_goal][1]
                    y_goal = goal_list[next_goal][0]
                    local_obstacle_flag = 1
                timer = 30
                
                print("Obstacle detected - obstacle avoidance")
                left_speed, right_speed, mean_prev = await obstacle_avoidance(left_speed, right_speed, mean_prev)

                left_angular = left_speed * speed_constant / wheel_radius
                right_angular = right_speed * speed_constant / wheel_radius
                    
            elif state == 2:
                print('Kidnapping')
                state = 10
                break

            speed_left = node["motor.left.speed"] 
            speed_right = node["motor.right.speed"] 

            ret, frame = cap.read()
            if not ret:
                pos_cam, ang_cam, cam_flag = [[-1,-1], -1, False]
                print("Unable to get frame")
            else:
                frame_height, frame_width = frame.shape[:2]
                frame = cv2.warpPerspective(frame, transformation_matrix, (frame_width, frame_height))
                frame = cv2.resize(frame, (640, 427)) # Resizing with 3/2 aspect ratio
                pos_cam, ang_cam, cam_flag = await cam_robot_pos(frame)
                if cam_flag:
                    pos_cam_pxl = map2frameCoord(((pos_cam + [70, 70])*(640/1260)).astype(int)) # Convert to pixels
                    cam_positions = np.vstack((cam_positions, pos_cam_pxl))

            x_est, P_est = await kalman_filter(speed_left, speed_right, pos_cam[0], pos_cam[1], ang_cam, cam_flag, x_est, P_est)

            x_est_coord = np.array([x_est[0][0], x_est[1][0]])
            pos_kalman_pxl = map2frameCoord(((x_est_coord + [70, 70])*(640/1260)).astype(int)) # Convert to pixels
            kalman_positions = np.vstack((kalman_positions, pos_kalman_pxl))

            # Draw trajectories on frame
            # Path
            for i in range(1, len(goal_list)):
                goal = goal_list[i]
                goal = [goal[1], goal[0]]
                goal_prev = goal_list[i - 1]
                goal_prev = [goal_prev[1], goal_prev[0]]
                goal_mm = np.array(goal, dtype=np.float64)*140
                goal_pxl = map2frameCoord(((goal_mm + [70, 70])*(640/1260)).astype(int))
                goal_mm_prev = np.array(goal_prev, dtype=np.float64)*140
                goal_pxl_prev = map2frameCoord(((goal_mm_prev + [70, 70])*(640/1260)).astype(int))
                cv2.line(frame, tuple(goal_pxl_prev), tuple(goal_pxl), (0, 0, 255), 2)
                
            # Camera
            for i in range(1, len(cam_positions)):
                cv2.line(frame, tuple(cam_positions[i - 1]), tuple(cam_positions[i]), (0, 255, 0), 2)

            # Kalman
            for i in range(1, len(kalman_positions)):
                cv2.line(frame, tuple(kalman_positions[i - 1]), tuple(kalman_positions[i]), (0, 255, 255), 2)

            # Highlight robot position
            if cam_flag:
                cv2.circle(frame, pos_cam_pxl, 5, (255,255,0),-1)


            cv2.imshow(win_name, frame)
            cv2.waitKey(1)
            
            time.sleep(delta_t)
            
    # Stop robot at the end    
    left_speed = 0
    right_speed = 0
    node.send_set_variables(motors(int(left_speed), int(right_speed)))

    return

#### Camera window legend
- Red line: Global path

- Yellow line: Path of the robot estimated by the Kalman filter

- Green line: Path of the robot estimated by the camera

- Cyan dot: instantaneous position of the robot estimated by the camera


In [ ]:
camera_index = 0
cap = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW) # Open camera

# Open window for visualization
win_name = "Camera"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(win_name, 640, 427)

# Initialize variables
MAG_REF_HSV = (172, 170, 100)
CYAN_REF_HSV = (90, 180, 70)
MAX_CIRCLE_DISTANCE = 50

transformation_matrix = map_init()

await main_control_loop(transformation_matrix)

Camera opened successfully.
Error: Expected 4 squares, but found 3
Couldn't find the map :(, attempt 1
Error: Expected 4 squares, but found 3
Couldn't find the map :(, attempt 2
Succes: 4 squares found !
Squares detected. Computing transformation matrix.
I am being kidnapped :(, put me on the ground to continue
I am being kidnapped :(, put me on the ground to continue
I am being kidnapped :(, put me on the ground to continue
I am being kidnapped :(, put me on the ground to continue
I am being kidnapped :(, put me on the ground to continue
I am being kidnapped :(, put me on the ground to continue
I am being kidnapped :(, put me on the ground to continue
I am being kidnapped :(, put me on the ground to continue
I am being kidnapped :(, put me on the ground to continue
I am being kidnapped :(, put me on the ground to continue
I am being kidnapped :(, put me on the ground to continue
I am being kidnapped :(, put me on the ground to continue
I am being kidnapped :(, put me on the ground to 

In [ ]:
cap.release()
cv2.destroyWindow(win_name)

## Conclusion
Despite successfully accomplishing the goals, there exist several areas of improvement for the project. For instance, the robot can only accurately follow the path at low speeds due to the PID response and the adjustment time. Another improvement lies in the obstacle avoidance logic. If the obstacle leads the robot towards a global obstacle, when it is done avoiding the local obstacle it will possibly go over the global obstacle. Finally, there exists an overshoot originating from the controller. When there are turns, the robot tends to change the angle too much causing very small oscillations to return to the desired direction. These areas of improvement, although important, did not significantly affect the performance of the robot, thus, the original goals set were met. Overall, we consider the code presented a robust implementation that allows the Thymio to follow a calculated optimal path along an obstacle-filled map thanks to a controller, while improving the accuracy of the estimation of its position with a filter and remaining perceptive of local obstacles. 

## Bibliography

PID Explained. (n.d.). PID Controller Explained. Retrieved from https://pidexplained.com/pid-controller-explained/

Martins, F. (n.d.). Lab 4: Robotics Simulation Labs. Retrieved November 22, 2024, from https://felipenmartins.github.io/Robotics-Simulation-Labs/Lab4/

Mondada, F. (2024). Code for A* Algorithm (Exercise 5). Provided in Mobile Robotics, EPFL. Unpublished class material.

Mondada, F. (2024). Code for Artificial Neural Networks and Motor Movement (Exercise 3). Provided in Mobile Robotics, EPFL. Unpublished class material.

Wikipedia contributors. (n.d.). HSL and HSV. Wikipedia. Retrieved December 5, 2024, from https://en.wikipedia.org/wiki/HSL_and_HSV

OpenCV. (n.d.). Image filtering in OpenCV. Retrieved December 5, 2024, from https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html

OpenCV. (n.d.). Contours in OpenCV. Retrieved December 5, 2024, from https://docs.opencv.org/3.4/d4/d73/tutorial_py_contours_begin.html

OpenCV. (n.d.). Hough Circle Transform in OpenCV. Retrieved December 5, 2024, from https://docs.opencv.org/3.4/d4/d70/tutorial_hough_circle.html

Stack Overflow. (n.d.). cv2.approxPolyDP, cv2.arcLength – How do these work? Retrieved December 5, 2024, from https://stackoverflow.com/questions/62274412/cv2-approxpolydp-cv2-arclength-how-these-works